In [1]:
import pandas as pd
from datetime import datetime, timedelta
from geopy.geocoders import Nominatim
pd.options.mode.copy_on_write = True 

In [2]:
#Lit le fichier des gares et filtrent sur les gares qui accueillent des voyageurs
liste_gare = pd.read_csv("./liste-des-gares.csv", sep=';')
liste_gare = liste_gare.loc[liste_gare["VOYAGEURS"] == "O"]

In [3]:
#Lit le fichier de stops
liste_stops_ter = pd.read_csv("./export-ter-gtfs-last/stops.csv", sep=",")
liste_stops_ter = liste_stops_ter[[str.strip(x[:18]) == "StopPoint:OCETrain" for x in liste_stops_ter["stop_id"]]]
# liste_stops_ter["CODE_UIC"] = [ int(x.split("-")[-1]) for x in liste_stops_ter["stop_id"]]
# liste_stops_ter = pd.merge(liste_stops_ter, liste_gare, how="left", on="CODE_UIC")
#liste_stops_ter["stop_name"] = [city.strip().upper().replace(" ", "-") for city in liste_stops_ter["stop_name"]]
# liste_stops_ter["city"] = [geolocator.reverse(str(row.stop_lat) + "," + str(row.stop_lon)).raw["address"]["town"].upper() for row in liste_stops_ter.itertuples()]
# i = 0
# for row in liste_stops_ter.itertuples():
#     if i > 5: break
#     i += 1
#     geolocator = Nominatim(user_agent="my_geopy_app")
#     location = geolocator.reverse(str(row.stop_lat) + "," + str(row.stop_lon))
#     print(location.raw["address"])
#     # row["city"] = location.raw["address"]

In [4]:
geolocator = Nominatim(user_agent="my_geopy_app")
location = geolocator.reverse("48.9523850,7.63409000")
location.raw["address"]

{'amenity': 'Amazon Hub',
 'road': 'Allée des Tilleuls',
 'village': 'Niederbronn-les-Bains',
 'municipality': 'Haguenau-Wissembourg',
 'county': 'Bas-Rhin',
 'ISO3166-2-lvl6': 'FR-67',
 'state': 'Grand Est',
 'ISO3166-2-lvl4': 'FR-GES',
 'region': 'France métropolitaine',
 'postcode': '67110',
 'country': 'France',
 'country_code': 'fr'}

In [5]:
# Lit le fichier des temps d'arrêts des trains
liste_stop_times_ter = pd.read_csv("./export-ter-gtfs-last/stop_times.csv", sep=",")

liste_stop_times_ter = liste_stop_times_ter[[str.strip(str(x)[:18]) == "StopPoint:OCETrain" for x in liste_stop_times_ter["stop_id"]]]
liste_stop_times_ter = pd.merge(liste_stop_times_ter, liste_stops_ter[["stop_id", "stop_name"]], how="inner", on="stop_id")

# col_trip_id_ter = liste_stop_times_ter["trip_id"].unique()
# liste_stop_times_ter["CODE_UIC"] = [ int(x.split("-")[-1]) for x in liste_stop_times_ter["stop_id"]]
# liste_stop_times_ter = pd.merge(liste_stop_times_ter, liste_gare, how="inner", on="CODE_UIC")
# liste_stop_times_ter.COMMUNE.fillna("error")
# liste_stop_times_ter.loc[liste_stop_times_ter["COMMUNE"] == "error"]

In [6]:
liste_stop_times_ter.columns

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled',
       'stop_name'],
      dtype='object')

In [7]:
liste_trips_ter = pd.read_csv("./export-ter-gtfs-last/trips.csv", sep=",")
liste_calendar_ter = pd.read_csv("./export-ter-gtfs-last/calendar_dates.csv", sep=",")
print(len(liste_trips_ter))

liste_trips_ter = pd.merge(liste_trips_ter, liste_calendar_ter[["service_id", "date"]], how="inner", on="service_id")

print(len(liste_trips_ter))

liste_stop_times_ter = pd.merge(liste_stop_times_ter, liste_trips_ter[["date", "trip_id"]], how="inner", on="trip_id")

print(len(liste_stop_times_ter))

52753
819405
5955580


In [8]:
liste_stop_times_ter.arrival_time.head(5)


0    05:00:00
1    05:08:00
2    05:30:00
3    05:40:00
4    05:49:00
Name: arrival_time, dtype: object

In [9]:
liste_stop_times_ter["date"] = [datetime(int(str(row.date)[:4]), int(str(row.date)[4:6]),  int(str(row.date)[-2:]), 0 if int(str(row.arrival_time).split(":")[0]) >= 24 else int(str(row.arrival_time).split(":")[0]), int(str(row.arrival_time).split(":")[1])) for row in liste_stop_times_ter.itertuples()]

In [10]:
liste_stop_times_ter.date.head(5)

0   2024-11-18 05:00:00
1   2024-11-18 05:08:00
2   2024-11-18 05:30:00
3   2024-11-18 05:40:00
4   2024-11-18 05:49:00
Name: date, dtype: datetime64[ns]

In [17]:
liste_stop_times_ter["stop_name"] = [name.upper() for name in liste_stop_times_ter["stop_name"]]

In [44]:
depart = "OBERNAI"
destination = "HAGUENAU"

def check_destination(destination, layer):
    return len(layer.loc[layer["stop_name"] == destination]) > 0

def path_back_propagation(layers, destination):
    paths = []
    city_to_fetch = destination
    layers[-1].loc[layers[-1]["stop_name"] == city_to_fetch].to_csv("data_cleaned/test.csv")
    for layer in reversed(layers):
        data = layer.loc[layer["stop_name"] == city_to_fetch]
        # print(len(data))
        if len(data) == 0:
            return "error"
        data = data.iloc[0]
        paths.append(data['current_path'])
        if data["previous_path"] == "":
            return paths[::-1]
        city_to_fetch = data["previous_path"].split(';')[0]
        # print(data["previous_path"])

def filter_direction_trip(data, namesList, start_date=False):
    find_name = False
    trip_id = ""
    end_date = ""
    default_value = False
    if start_date != False:
        end_date = start_date + timedelta(days=1)
        default_value = True
    data = data.sort_values(by=["trip_id", "stop_sequence"])
    for index, row in data.iterrows():
        if not default_value:
            date_str = row.previous_path.split(';')[-1]
            date = datetime.strptime(date_str, "%d/%m/%Y, %H:%M:%S")
            start_date = date + timedelta(minutes=5)
            end_date = start_date + timedelta(days=1)

        if row.trip_id != trip_id:
            trip_id = row.trip_id
            find_name = False
        if row.stop_name in namesList and (row.date >= start_date and row.date <= end_date):
            find_name = True
        data.loc[index, 'isDirectionOk'] = "ok" if find_name else "ko"
    data.to_csv("data_cleaned/test-ok.csv")
    data = data.drop(data[data.isDirectionOk == "ko"].index)
    data.to_csv("data_cleaned/test-drop.csv")
    return data

def shortest_path(depart, destination, start_date):
    layers = []
    not_founded = 0 # vérifie qu'on a pas trouvé un chemin vers l'arrivée
    liste_stop_times_ter["isDirectionOk"] = ""

    start_date = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
    end_date = start_date + timedelta(days=1)
    date_mask = (liste_stop_times_ter["date"] > start_date) & (liste_stop_times_ter["date"] < end_date)

    trips_id = set(liste_stop_times_ter.loc[(liste_stop_times_ter["stop_name"] == depart) & date_mask]["trip_id"])
    layer = liste_stop_times_ter.loc[(liste_stop_times_ter["trip_id"].isin(trips_id)) & date_mask]
    
    layer["previous_path"] = ""
    layer = filter_direction_trip(layer, [depart], start_date)  
    layer = layer.sort_values(by="date").drop_duplicates(subset=["stop_id"])

    
    layer["current_path"] = layer["trip_id"] + ";" + layer["stop_name"] + ";" + layer["date"].dt.strftime("%d/%m/%Y, %H:%M:%S")

    layers.append(layer)
    layer.to_csv("data_cleaned/layer_1.csv")

    while not_founded < 100:
        if check_destination(destination, layers[-1]):
            print(path_back_propagation(layers, destination))
            return "destination trouver"
        
        filtered_data = liste_stop_times_ter.loc[(liste_stop_times_ter["stop_name"].isin(layers[-1]["stop_name"].unique())) & date_mask]

        paths = {}
        for row in filtered_data.itertuples():
            paths[row.trip_id] = str(row.stop_name) + ";" + row.date.strftime("%d/%m/%Y, %H:%M:%S")
        # start_date = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
        # end_date = start_date + timedelta(days=1)
        # date_mask = (liste_stop_times_ter["date"] > start_date) & (liste_stop_times_ter["date"] < end_date)
        
        trips_id = set(filtered_data["trip_id"]) - set(layers[-1]["trip_id"])
        print(len(layers[-1]["stop_name"].unique()))
        layer = liste_stop_times_ter.loc[(liste_stop_times_ter["trip_id"].isin(trips_id)) & date_mask]
        
        layer["previous_path"] = [paths[data.trip_id] for data in layer.itertuples()]
        layer.to_csv("data_cleaned/final_layer-2.csv")
        print(layers[-1]["stop_name"].unique())
        layer = filter_direction_trip(layer, layers[-1]["stop_name"].unique())
        layer.to_csv("data_cleaned/final_layer-4.csv")

        layer = layer.sort_values(by="date").drop_duplicates(subset=["stop_id"])
        layer.to_csv("data_cleaned/final_layer-3.csv")
        
        
        layer["current_path"] = layer["trip_id"] + ";" + layer["stop_name"] + ";" + layer["date"].dt.strftime("%d/%m/%Y, %H:%M:%S")
        layer.to_csv("data_cleaned/final_layer.csv")
        layers.append(layer)
        
        not_founded += 1
    return "not found excedeed 100 search"
shortest_path(depart, destination, "2024-11-19 05:00:00")
# trips_id = set(liste_stop_times_ter.loc[liste_stop_times_ter["COMMUNE"] == depart]["trip_id"].unique())
# print(check_destination(destination,liste_stop_times_ter[liste_stop_times_ter["trip_id"].isin(trips_id)]))

20
['OBERNAI' 'GOXWILLER' 'BISCHOFFSHEIM' 'ROSHEIM' 'GERTWILLER' 'BARR'
 'DORLISHEIM' 'MOLSHEIM' 'DACHSTEIN' 'DUTTLENHEIM' 'DUPPIGHEIM'
 'ENTZHEIM AÉROPORT' 'LINGOLSHEIM' 'STRASBOURG ROETHIG' 'STRASBOURG'
 'EICHHOFFEN' 'EPFIG' 'DAMBACH-LA-VILLE' 'SCHERWILLER' 'SÉLESTAT']
['OCESN831701F3073049:2024-10-11T00:34:04Z;STRASBOURG;19/11/2024, 06:06:00', 'OCESN832902F3468357:2024-10-11T00:34:04Z;HAGUENAU;19/11/2024, 05:57:00']


'destination trouver'

In [12]:

date = datetime.strptime("19/11/2024, 08:27:00", "%d/%m/%Y, %H:%M:%S")
print(date)
start_date = date + timedelta(minutes=5)
print(start_date)
end_date = start_date + timedelta(days=1)
print(end_date)

2024-11-19 08:27:00
2024-11-19 08:32:00
2024-11-20 08:32:00
